In [3]:
import cv2
import numpy as np
import tensorflow as tf
import os

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

In [4]:
tf.__version__

'2.4.1'

In [5]:
model = tf.keras.models.load_model("facenet_keras.h5")

In [6]:
#Carregar modelo SVM
import pickle
filename = 'SVC_model.sav'
modelSVC = pickle.load(open(filename, 'rb'))

In [7]:
def get_embedding(model, face):
    faces =[]
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    
    return yhat

In [8]:
detectorFace = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
largura, altura = 160, 160
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
camera = cv2.VideoCapture(0)

while (True):
    conectado, imagem = camera.read()
    imagemFace = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
    facesDetectadas = detectorFace.detectMultiScale(imagemFace, scaleFactor=1.5, minSize=(30,30))
    for (x, y, l, a) in facesDetectadas:
        imagemFace1 = cv2.resize(imagemFace[y:y + a, x:x + l], (largura, altura))
        cv2.rectangle(imagem, (x, y), (x + l, y + a), (0,255,0), 2)
        
        #embedding
        face = get_embedding(model, imagemFace1)

        
        #Normilize
        in_encoder = Normalizer()
        emdTrainX_norm = in_encoder.transform(face)
        
        #Predict
        yhat = modelSVC.predict(emdTrainX_norm)
        
        #id, confianca = reconhecedor.predict(imagemFace)
        nome = ""
        if yhat[0] == 0:
            nome = 'Renato'
        elif yhat[0] == 1:
            nome = 'Leticia'
        elif yhat[0] == 2:
            nome = 'Natalia'
        else :
            nome = 'Zé Mané'
            
        cv2.putText(imagem, nome, (x,y +(a+30)), font, 2, (0,255,0))

    cv2.imshow("Face", imagem)
    if cv2.waitKey(1) == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()